<a href="https://colab.research.google.com/github/JulieMew/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/LS_DS_211_assignment_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 1*

---

# Regression 1

## Assignment

You'll use another **New York City** real estate dataset. 

But now you'll **predict how much it costs to rent an apartment**, instead of how much it costs to buy a condo.

The data comes from renthop.com, an apartment listing website.

- [ ] Look at the data. Choose a feature, and plot its relationship with the target.
- [ ] Use scikit-learn for linear regression with one feature. You can follow the [5-step process from Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/05.02-introducing-scikit-learn.html#Basics-of-the-API).
- [ ] Define a function to make new predictions and explain the model coefficient.
- [ ] Organize and comment your code.

> [Do Not Copy-Paste.](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit) You must type each of these exercises in, manually. If you copy and paste, you might as well not even do them. The point of these exercises is to train your hands, your brain, and your mind in how to read, write, and see code. If you copy-paste, you are cheating yourself out of the effectiveness of the lessons.

If your **Plotly** visualizations aren't working:
- You must have JavaScript enabled in your browser
- You probably want to use Chrome or Firefox
- You may need to turn off ad blockers
- [If you're using Jupyter Lab locally, you need to install some "extensions"](https://plot.ly/python/getting-started/#jupyterlab-support-python-35)

## Stretch Goals
- [ ] Do linear regression with two or more features.
- [ ] Read [The Discovery of Statistical Regression](https://priceonomics.com/the-discovery-of-statistical-regression/)
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapter 2.1: What Is Statistical Learning?

In [1]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [17]:
# Read New York City apartment rental listing data
%matplotlib inline
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
assert df.shape == (49352, 34)

In [3]:
# Remove outliers: 
# the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= 1375) & (df['price'] <= 15500) & 
        (df['latitude'] >=40.57) & (df['latitude'] < 40.99) &
        (df['longitude'] >= -74.1) & (df['longitude'] <= -73.38)]

In [4]:
# Check dataframe to make sure it loaded in correctly.

print(df.shape)
df.head()

(48818, 34)


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
guess = df['price'].mean()
guess

3830.174035500081

In [30]:
errors = guess - df['price']
errors

0         830.174036
1       -1634.825964
2         980.174036
3         555.174036
4         480.174036
            ...     
49347     630.174036
49348    -119.825964
49349    1235.174036
49350     480.174036
49351    1630.174036
Name: price, Length: 49352, dtype: float64

In [33]:
mae = errors.abs().mean()
mae

1549.642448727267

In [ ]:
import plotly.express as px
px.scatter(df, x='GROSS_SQUARE_FEET', y='price')

In [5]:
# 1. Import the appropriate estimator class from Scikit-Learn

from sklearn.linear_model import LinearRegression

In [6]:
LinearRegression

sklearn.linear_model._base.LinearRegression

In [7]:
# 2. Instantiate this class

model = LinearRegression()

In [8]:
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
# 3. Arrange X features matrix & y target vector

df['latitude'], df['longitude']

(0        40.7145
 1        40.7947
 2        40.7388
 3        40.7539
 4        40.8241
           ...   
 49347    40.7426
 49348    40.7102
 49349    40.7601
 49350    40.7066
 49351    40.8699
 Name: latitude, Length: 48818, dtype: float64, 0       -73.9425
 1       -73.9667
 2       -74.0018
 3       -73.9677
 4       -73.9493
           ...   
 49347   -73.9790
 49348   -74.0163
 49349   -73.9900
 49350   -74.0101
 49351   -73.9172
 Name: longitude, Length: 48818, dtype: float64)

In [10]:
# Standard approach that can generalize to other problems

features = ['longitude', 'latitude']
target = ['price']

x_train = df[features]
y_train = df[target]

In [11]:
x_train.describe()

,longitude,latitude
count,48818.000000,48818.000000
mean,-73.972758,40.750756
std,0.028887,0.038962
min,-74.087300,40.571200
25%,-73.991800,40.728300
50%,-73.978000,40.751700
75%,-73.955000,40.774000
max,-73.700100,40.989400


In [12]:
y_train.describe()

,price
count,48818.000000
mean,3579.560982
std,1762.420875
min,1375.000000
25%,2500.000000
50%,3150.000000
75%,4095.000000
max,15500.000000


In [13]:
# 4. Fit the model

model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [44]:
# 5. Apply the model to new data
y_pred = model.predict(df[features])
y_pred

array([[2999.53175498],
       [3583.65896249],
       [4026.50603821],
       ...,
       [3883.51604908],
       [4086.65647434],
       [2950.59999349]])